In [144]:
import xlrd
import csv
import fnmatch
import os
from collections import Counter

DATA_DIR = '../../Data'

In [145]:
def remove_suffix(text, suffix):
    '''Remove a suffix from a string, returning a new string.'''
    if text.endswith(suffix):
        return text[:-len(suffix)]
    return text

In [146]:
def extract_data_table_from_sheet(sheet):
    '''Extract the most likely data table from given sheet.
    
    Use this on Excel data sheets from the CFG database.
    
    This method assumes a few things. The data table is most likely to be on the left-most side of the sheet,
    and contains 5-6 headers, which are generally "Number", "Glycan", "Average", "Stdev", "SEM", "%CV".
    
    Args:
        sheet (xlrd.sheet.Sheet): An xlrd Sheet object.
    Returns:
        list: A list containing cell coordinates for each of the 6 possible data columns: 
        Chart number, Glycan structure, Average RFU, StDev, SEM, % CV.
    '''
    headers = (("glycan no.", "chart number", "glycan number", "chart #"),
               ("structure on masterlist", "glycan - spacer", "glycan name", "structure", "glycan structure",
                "masterlist name", "iupac glycan name"),
               ("average rfu", "avg. rfu", "average", "rfu", "avg w/o max & min", "averagerfu",
                "avg w/o min/max", "average w/o max and min", "avgmeans-b w/o min/max", "avg rfu"),
               ("stdev", "stdev w/o max & min", "stdev w/o min/max", "stdev w/o max and min", "stddev"),
               ("sem", "sem w/o max & min", "sem w/o min/max", "sem w/o max and min"),
               ("% cv", "cv", "%cv")
               )
    header_locations = [None] * 6
    row_start, col_start = None, None
    # Find location of table by examining the first two table headers.
    # If these are next to each other, then we have found the table.
    for i, row in enumerate(sheet.get_rows()):
        for j, col in enumerate(row):
            if str(col.value).lower() in headers[0] and str(row[j+1].value).lower() in headers[1]:
                row_start, col_start = i, j
                break
    # If we can't find the first two table headers, then we presume the sheet doesn't contain the data
    # we want.
    if row_start is None or col_start is None:
        return header_locations
    # Otherwise, we use the position of the first two headers to identify the location
    # of the RFU, Stdev, SEM and % CV columns (if they exist)
    header_locations[0] = (row_start, col_start)
    header_locations[1] = (row_start, col_start+1)
    positions = [2,3,4,5]
    for header in headers[2:]:
        for i, col in enumerate(positions):
            if col_start + col >= sheet.ncols:
                continue
            if str(sheet.cell(row_start, col_start + col).value).lower() in header:
                header_locations[col] = (row_start, col_start + col)
                # There should be only one column for each type of data.
                positions.pop(i)
                continue
                
    # Now, check that there aren't any non-empty fields that haven't been matched
    # to a known data colum (as defined by our list of possible headers).
    if header_locations[0]:
        for i in range(6):
            if header_locations[0][1] + i >= sheet.ncols:
                continue
            if sheet.cell(header_locations[0][0], header_locations[0][1] + i).value and not header_locations[i]:
                print("Not matched!!")
                print(sheet.row(header_locations[0][0]))
                
    return header_locations

In [147]:
def get_data_sheet_name(workbook):
    '''Returns the most likely sheet name for a data-containing sheet,
    otherwise return None.
    
    Data sheets either contain the text "PA Result" in the sheet name,
    or start with the word "Core".
    
    Args:
        workbook (xlrd.workbook.Workbook): An xlrd workbook object.
    Returns:
        string: The name of the likely sheet to contain data.'''
    sheet_names = workbook.sheet_names()
    for sheet in sheet_names:
        if 'PA Result' in sheet:
            return sheet
    for sheet in sheet_names:
        if sheet.startswith('Core'):
            return sheet
    return None

In [136]:
data_dir = os.path.join(DATA_DIR, 'CFG_Data_Files/')
output_dir = os.path.join(DATA_DIR, 'CFG_Data_Files/')


# Get list of all data files within data directory (only excel files)
files = []
for file in os.listdir(data_dir):
    if fnmatch.fnmatch(file, '*.xls'):
        files.append(file)

# Pull list of sheet names for each file, so we can do a bit of exploratory analysis.
sheet_names_list = []
for file in files:
    workbook = xlrd.open_workbook(os.path.join(data_dir, file), on_demand=True)
    sheet_names = workbook.sheet_names()
    sheet_names_list.append(sheet_names)

# A tentative filter to select sheet names likely to contain data fields.
Counter([len([y for y in x if y.startswith('Core') or "PA Result" in y]) for x in sheet_names_list])

WARNING *** file size (2855937) not 512 + multiple of sector size (512)
WARNING *** file size (4458497) not 512 + multiple of sector size (512)
WARNING *** file size (4443137) not 512 + multiple of sector size (512)
WARNING *** file size (2955265) not 512 + multiple of sector size (512)
WARNING *** file size (4445185) not 512 + multiple of sector size (512)
WARNING *** file size (4214273) not 512 + multiple of sector size (512)
WARNING *** file size (4477953) not 512 + multiple of sector size (512)
WARNING *** file size (4506113) not 512 + multiple of sector size (512)
WARNING *** file size (3701761) not 512 + multiple of sector size (512)
WARNING *** file size (5082113) not 512 + multiple of sector size (512)
WARNING *** file size (4484097) not 512 + multiple of sector size (512)
WARNING *** file size (3400705) not 512 + multiple of sector size (512)
WARNING *** file size (4506625) not 512 + multiple of sector size (512)
WARNING *** file size (5063681) not 512 + multiple of sector siz

In [ ]:
# Just a quick look at files that seem to have multiple sheet names that might indicate they contain the
# relevant data.
# Note: Files that have doubled-up sheets with a (2) at the end seem to have the same content in each sheet. 
# Just take from one.

[(i, x) for i, x in enumerate(sheet_names_list) if len([y for y in x if y.startswith('Core') or "PA Result" in y]) > 1]

In [170]:
# Sheet names for all files that don't meet the inital filter
# We might want to look at these a bit closer and make sure we haven't missed anything.
missed_files = [(files[i], x) for i, x in enumerate(sheet_names_list) if 
                len([y for y in x if y.startswith('Core') or "PA Result" in y]) == 0]

[x for x in missed_files if '10 uM' not in ''.join(x[1]) and '10uM' not in ''.join(x[1])]

# Note: Some files look like they might be ok, with just a single sheet.
# So maybe we can also select a single sheet if that file just contains one sheet.

[('Summary_DE274_whole_14599_PA_Results.xls', ['5K', '10K', '20K', '50K']),
 ('Marchantia 16kDa-EVD-021720-FITC_060705_1316 PA Result.xls',
  ['Marchantia 16kDa-EVD-021720-FIT']),
 ('Results_Quail_HK_G1_1997_14532_V5_PA_Results.xls',
  ['32000HAU', '16000HAU', '6400HAU', '3200HAU']),
 ('A_Mem_14_98_14176_V5_PA_Results.xls',
  ['1600HAU', '800HAU', '4000HAU', '8000HAU']),
 ('Marasmius oreades lectin 050704.xls', ['Sheet1', '050704-348', 'V.2.8']),
 ('WT StcE (anit-C-term StcE) 031404-561.xls', ['031405-561', 'V3']),
 ('E. coli UTI89.xls', ['E. coli UTI89']),
 ('Memphis-7_1994_10X_5X_neat_15065_v5_PA_Results.xls',
  ['80HAU', '160HAU', 'neat']),
 ('hSiglec7-Fc.xls', ['101504-449-hSig7', 'V3']),
 ('OK_323_2003_100X_20X_5X_15078_v5_PA_Results.xls',
  ['320HAU', '1600HAU', '6400HAU', 'neat']),
 ('Summary_Gull_whole_14571_PA_Results.xls', ['5K', '10K', '20K', '50K']),
 ('HK_4167_99_14451_v5.0_PA_results_summary.xls', ['20X', '10X', '4X']),
 ('OK_1992_2005_15079_100X_20X_5X_v5_PA_Results.xls'

In [ ]:
# Extract location of data columns from all files.
errors = []
files_with_data = []
for file in files:
    workbook = xlrd.open_workbook(os.path.join(data_dir, file), on_demand=True)
    sheet_name = get_data_sheet_name(workbook)
    if sheet_name:
        sheet = workbook.sheet_by_name(sheet_name)
        headers = extract_data_table_from_sheet(sheet)
        if headers[0] is None or headers[1] is None:
            errors.append([file, list(sheet.get_rows())])
        else:
            files_with_data.append([file, sheet_name, headers])

In [159]:
# Extract data into csv files for easy manipulation later
for file, sheetname, indices in files_with_data:
    workbook = xlrd.open_workbook(os.path.join(data_dir, file))
    sheet = workbook.sheet_by_name(sheetname)
    # Only want the first 6 columns for now.
    data = [["Chart Number", "Structure", "RFU", "StDev", "SEM", "CV"]]
    nrows = sheet.nrows
    header_row = indices[0][0]
    for j in range(header_row + 1, nrows):
        col_indices = [x[1] if x is not None else None for x in indices]
        data_line = [sheet.cell_value(j, col_index) if col_index is not None else None for col_index in col_indices]
        # Sometimes they put spacer designations below the main data table - this is a crude way to identify the end of
        # the original table.
        if data_line[0] == '' and data_line[1] == '':
            break
        data.append(data_line)
    #Convert row number to int.
    for x in data[1:]:
        if x[0] is not None:
            x[0] = int(x[0])
    output_name = file.replace('.xls', '.csv')
    with open(os.path.join(output_dir, output_name), 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)

WARNING *** file size (2855937) not 512 + multiple of sector size (512)
WARNING *** file size (4458497) not 512 + multiple of sector size (512)
WARNING *** file size (4443137) not 512 + multiple of sector size (512)
WARNING *** file size (2955265) not 512 + multiple of sector size (512)
WARNING *** file size (4445185) not 512 + multiple of sector size (512)
WARNING *** file size (4214273) not 512 + multiple of sector size (512)
WARNING *** file size (4477953) not 512 + multiple of sector size (512)
WARNING *** file size (4506113) not 512 + multiple of sector size (512)
WARNING *** file size (5082113) not 512 + multiple of sector size (512)
WARNING *** file size (4484097) not 512 + multiple of sector size (512)
WARNING *** file size (3400705) not 512 + multiple of sector size (512)
WARNING *** file size (4506625) not 512 + multiple of sector size (512)
WARNING *** file size (5063681) not 512 + multiple of sector size (512)
WARNING *** file size (5073921) not 512 + multiple of sector siz